# 11 协议对比与选型

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/forhow134/ai-coding-guide/blob/main/demos/11-protocols/protocols_overview.ipynb)

**预计 API 费用: ~$0**

本 Notebook 对比 MCP、A2A、ANP 三大协议,并演示选型逻辑。

## Experiment 1: 三大协议核心Comparison
<!-- 实验 1: 三大协议核心对比 -->

In [ ]:
import pandas as pd

# 三大协议对比表
comparison_data = {
    "维度": ["目标", "场景", "规模", "通信", "状态", "发现", "路由", "容错", "成熟度"],
    "MCP": [
        "AI 调工具",
        "单个 AI + 外部工具",
        "小(几个到几十个工具)",
        "JSON-RPC(同步)",
        "无状态",
        "无(预定义)",
        "无",
        "无",
        "⭐⭐⭐⭐⭐ 成熟"
    ],
    "A2A": [
        "Agent 调 Agent",
        "多 Agent 协作",
        "中(几个到几十个 Agent)",
        "REST API(异步)",
        "有任务状态",
        "Agent Card",
        "手动指定",
        "需自己实现",
        "⭐⭐⭐ 早期"
    ],
    "ANP": [
        "大规模 Agent 网络",
        "企业级 Agent 平台",
        "大(成百上千 Agent)",
        "分布式(异步)",
        "有 Agent 状态",
        "Registry",
        "智能路由",
        "内置",
        "⭐⭐ 提案"
    ]
}

df = pd.DataFrame(comparison_data)
print("=== MCP vs A2A vs ANP ===")
print(df.to_string(index=False))

## Experiment 2: A2A 协作模式演示
<!-- 实验 2: A2A 协作模式演示 -->

In [ ]:
# Mock/Simulate A2A 协作流程

import json
from typing import Dict, Any

# Mock/Simulate Agent Card
translator_card = {
    "name": "translator_agent",
    "description": "专业翻译 Agent",
    "capabilities": [
        {
            "name": "translate",
            "description": "翻译文本",
            "input_schema": {
                "type": "object",
                "properties": {
                    "text": {"type": "string"},
                    "from": {"type": "string"},
                    "to": {"type": "string"}
                }
            }
        }
    ]
}

reviewer_card = {
    "name": "reviewer_agent",
    "description": "文本校对 Agent",
    "capabilities": [
        {
            "name": "review",
            "description": "校对文本",
            "input_schema": {
                "type": "object",
                "properties": {
                    "text": {"type": "string"},
                    "language": {"type": "string"}
                }
            }
        }
    ]
}

print("=== A2A 协作演示 ===")
print("\n1️⃣ 翻译 Agent Card:")
print(json.dumps(translator_card, indent=2, ensure_ascii=False))

print("\n2️⃣ 校对 Agent Card:")
print(json.dumps(reviewer_card, indent=2, ensure_ascii=False))

# Mock/Simulate任务流程
print("\n3️⃣ 任务流程:")
print("用户输入 → 翻译 Agent → 校对 Agent → 最终输出")

# Create翻译任务
translate_task = {
    "id": "task_001",
    "agent_id": "translator_agent",
    "capability": "translate",
    "input": {
        "text": "Hello, world!",
        "from": "en",
        "to": "zh"
    },
    "status": "running"
}

print("\n4️⃣ 翻译任务:")
print(json.dumps(translate_task, indent=2, ensure_ascii=False))

# Mock/Simulate翻译结果
translate_artifact = {
    "task_id": "task_001",
    "status": "completed",
    "parts": [
        {
            "type": "text",
            "content": "你好,世界!"
        }
    ]
}

print("\n5️⃣ 翻译结果:")
print(json.dumps(translate_artifact, indent=2, ensure_ascii=False))

# Create校对任务
review_task = {
    "id": "task_002",
    "agent_id": "reviewer_agent",
    "capability": "review",
    "input": {
        "text": translate_artifact["parts"][0]["content"],
        "language": "zh"
    },
    "status": "running"
}

print("\n6️⃣ 校对任务:")
print(json.dumps(review_task, indent=2, ensure_ascii=False))

# Mock/Simulate校对结果
review_artifact = {
    "task_id": "task_002",
    "status": "completed",
    "parts": [
        {
            "type": "text",
            "content": "你好,世界!"
        },
        {
            "type": "metadata",
            "content": {
                "issues_found": 0,
                "confidence": 1.0
            }
        }
    ]
}

print("\n7️⃣ 校对结果:")
print(json.dumps(review_artifact, indent=2, ensure_ascii=False))

## Experiment 3: ANP 路由策略演示
<!-- 实验 3: ANP 路由策略演示 -->

In [ ]:
# Mock/Simulate ANP 路由逻辑

from typing import List, Dict
import random

# Mock/Simulate Agent 注册信息
class AgentInfo:
    def __init__(self, agent_id: str, capability: str, load: float, region: str):
        self.agent_id = agent_id
        self.capability = capability
        self.load = load
        self.region = region
        self.status = "online"
    
    def __repr__(self):
        return f"Agent({self.agent_id}, load={self.load:.2f}, region={self.region})"

# Mock/Simulate Registry
registry = [
    AgentInfo("translator_001", "translate", 0.3, "us-west"),
    AgentInfo("translator_002", "translate", 0.6, "us-east"),
    AgentInfo("translator_003", "translate", 0.1, "asia"),
    AgentInfo("translator_004", "translate", 0.9, "europe"),
]

print("=== ANP 路由策略演示 ===")
print("\n可用 Agent:")
for agent in registry:
    print(f"  {agent}")

# 路由策略 1: 最低负载
def route_least_load(agents: List[AgentInfo]) -> AgentInfo:
    return min(agents, key=lambda a: a.load)

print("\n1️⃣ 最低负载策略:")
selected = route_least_load(registry)
print(f"  选择: {selected}")

# 路由策略 2: 地理位置最近
def route_geo_proximity(agents: List[AgentInfo], user_region: str) -> AgentInfo:
    # 简化版:直接匹配 region
    for agent in agents:
        if agent.region == user_region and agent.load < 0.8:
            return agent
    return min(agents, key=lambda a: a.load)

print("\n2️⃣ 地理位置策略(用户在 asia):")
selected = route_geo_proximity(registry, "asia")
print(f"  选择: {selected}")

# 路由策略 3: Round Robin
round_robin_index = 0
def route_round_robin(agents: List[AgentInfo]) -> AgentInfo:
    global round_robin_index
    available = [a for a in agents if a.load < 0.8]
    if not available:
        available = agents
    selected = available[round_robin_index % len(available)]
    round_robin_index += 1
    return selected

print("\n3️⃣ Round Robin 策略:")
for i in range(3):
    selected = route_round_robin(registry)
    print(f"  第 {i+1} 次: {selected}")

## Experiment 4: 协议选型决策树
<!-- 实验 4: 协议选型决策树 -->

In [ ]:
# 协议选型决策逻辑

def select_protocol(scenario: Dict[str, Any]) -> str:
    """
    根据场景自动选择合适的协议
    """
    needs_tools = scenario.get("needs_tools", False)
    num_agents = scenario.get("num_agents", 1)
    needs_routing = scenario.get("needs_routing", False)
    needs_fault_tolerance = scenario.get("needs_fault_tolerance", False)
    
    # 决策逻辑
    if num_agents == 1 and needs_tools:
        return "MCP (单个 AI 需要外部工具)"
    
    if num_agents > 1 and num_agents <= 10:
        return "A2A (少量 Agent 协作)"
    
    if num_agents > 10 or needs_routing or needs_fault_tolerance:
        return "ANP (大规模 Agent 网络)"
    
    return "不需要协议"

# Test案例
test_cases = [
    {
        "name": "个人 AI 助手",
        "needs_tools": True,
        "num_agents": 1,
        "needs_routing": False,
        "needs_fault_tolerance": False
    },
    {
        "name": "内容创作平台",
        "needs_tools": True,
        "num_agents": 3,
        "needs_routing": False,
        "needs_fault_tolerance": False
    },
    {
        "name": "企业 Agent 平台",
        "needs_tools": True,
        "num_agents": 100,
        "needs_routing": True,
        "needs_fault_tolerance": True
    },
    {
        "name": "客服系统",
        "needs_tools": True,
        "num_agents": 5,
        "needs_routing": False,
        "needs_fault_tolerance": False
    }
]

print("=== 协议选型决策树 ===")
print()

for case in test_cases:
    name = case.pop("name")
    recommendation = select_protocol(case)
    print(f"场景: {name}")
    print(f"  Agent 数量: {case['num_agents']}")
    print(f"  需要路由: {case['needs_routing']}")
    print(f"  需要容错: {case['needs_fault_tolerance']}")
    print(f"  推荐协议: {recommendation}")
    print()

## Experiment 5: 协议混用Example
<!-- 实验 5: 协议混用示例 -->

In [ ]:
# 企业级场景:三层协议混用

print("=== 企业级 Agent 平台架构 ===")
print()
print("用户请求:分析本季度销售数据")
print()
print("执行流程:")
print()
print("┌─ ANP (网络层)────────────────────┐")
print("│ 1. API Gateway 接收请求           │")
print("│ 2. Router 查询 Registry           │")
print("│ 3. Load Balancer 选择 Agent       │")
print("└──────────────────────────────────┘")
print("          ↓")
print("┌─ A2A (Agent 层)─────────────────┐")
print("│ 4. 数据分析 Agent 接收任务        │")
print("│ 5. 协调 3 个子 Agent:             │")
print("│    - 数据清洗 Agent               │")
print("│    - 统计分析 Agent               │")
print("│    - 可视化 Agent                 │")
print("└──────────────────────────────────┘")
print("          ↓")
print("┌─ MCP (工具层)───────────────────┐")
print("│ 6. 数据清洗 Agent:                │")
print("│    MCP → PostgreSQL Server        │")
print("│ 7. 统计分析 Agent:                │")
print("│    MCP → Python Executor          │")
print("│ 8. 可视化 Agent:                  │")
print("│    MCP → Chart Generator          │")
print("└──────────────────────────────────┘")
print("          ↓")
print("返回用户:分析报告 + 图表")
print()
print("关键洞察:")
print("  • ANP 提供:路由、负载均衡、容错")
print("  • A2A 提供:Agent 间协作、任务传递")
print("  • MCP 提供:外部工具调用能力")
print("  • 三层协议互补,不冲突!")

## Hands-on Exercises
<!-- 动手练习 -->

1. **设计你的 Agent 架构**: 根据你的项目需求,选择合适的协议组合
2. **实现 A2A 协调器**: 编写代码协调多个 Agent
3. **实现简单的 ANP Router**: 实现负载均衡逻辑
4. **对比实际性能**: 测试单 Agent vs 多 Agent 的效率差异

---

## Key Takeaways
<!-- 关键要点总结 -->

1. **三层模型**: MCP(工具层) → A2A(Agent 层) → ANP(网络层)
2. **不是竞争关系**: 每一层解决不同规模的问题
3. **选型原则**: 从需求出发,不要过度设计
4. **可以混用**: 大型项目同时使用多个协议
5. **成熟度不同**: MCP 成熟,A2A 早期,ANP 提案

---

**下一步**: 进入 [第 12 章:RAG & 记忆存储](../../docs/12-rag-memory/index.md),学习如何让 AI 记住信息。